In [1]:
# Load IPFX libraries
from ipfx.dataset.create import create_ephys_data_set
from ipfx.data_set_features import extract_data_set_features
from ipfx.utilities import drop_failed_sweeps

# Load pandas library too
import pandas as pd
mouse_data_dict = {}

In [2]:
from ipfx.data_access import get_archive_info
archive_url, file_manifest, experiment_metadata = get_archive_info(dataset="mouse")
mouse_specimen_id = list(experiment_metadata['cell_specimen_id'])

In [3]:
def summarize_cell_ephys_features(lsa_results):
    hero_keep_features = ['adapt', 'avg_rate', 'first_isi', 'isi_cv', 'latency', 'mean_isi', 'median_isi', 
                          'stim_amp']
    rheo_keep_features = ['threshold_v', 'peak_v', 'trough_v', 
     'fast_trough_v', 'adp_v', 'width', 'upstroke_downstroke_ratio', 'peak_t', 'fast_trough_t', 'trough_t']
    
    overall_cell_keep_features = ['v_baseline', 'rheobase_i', 'fi_fit_slope', 
                                  'sag', 'vm_for_sag', 'input_resistance', 'tau']
    
    hero_small_dict = lsa_results['hero_sweep'][hero_keep_features]
    rheobase_sweep_index = lsa_results['rheobase_sweep'].name
    rheobase_sweep = lsa_results['spikes_set'][rheobase_sweep_index].iloc[0]
    
    rheo_spike_small_dict = rheobase_sweep[rheo_keep_features]
    rheo_first_isi = lsa_results['rheobase_sweep']['first_isi']
    rheo_spike_small_dict['rheo_first_isi'] = rheo_first_isi

    spike_comb_dict = {**hero_small_dict, **rheo_spike_small_dict}

    overall_cell_features = {x: lsa_results[x] for x in overall_cell_keep_features if x in lsa_results}
    final_cell_feature_dict = {**spike_comb_dict, **overall_cell_features}
    return(final_cell_feature_dict)

In [ ]:
import urllib.request
from ipfx.dataset.create import create_ephys_data_set
from ipfx.utilities import drop_failed_sweeps

for id in human_specimen_id[145:]:
    specimen_id = id
    row = file_manifest.query('cell_specimen_id == {} & technique == "intracellular_electrophysiology"'.format(specimen_id))
    archive_uri = row['archive_uri'].values[0]
    file_name = row['file_name'].values[0]
    print('Intrinsic ephys file url: ' + archive_uri)
    print('Intrinsic ephys file name: ' + file_name)

    print('downloading data for cell from DANDI...')
    try:
        urllib.request.urlretrieve(archive_uri, file_name)
        print('data downloaded!')

        # Create Ephys Data Set

        print('loading dataset into data structure...')
        #nwb_file_name = '/Users/stripathy/Downloads/sub-601462951_ses-601810292_icephys.nwb'
        data_set = create_ephys_data_set(nwb_file=file_name) ##loads nwb file into ipfx data structure

        # Drop failed sweeps: sweeps with incomplete recording or failing QC criteria
        drop_failed_sweeps(data_set)

        from ipfx.feature_extractor import SpikeFeatureExtractor, SpikeTrainFeatureExtractor

        import ipfx.stimulus_protocol_analysis as spa
        from ipfx.epochs import get_stim_epoch
        import matplotlib.pyplot as plt


        # get sweep table of Long Square sweeps
        long_square_table = data_set.filtered_sweep_table(
            stimuli=data_set.ontology.long_square_names
        )
        long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)

        # Select epoch corresponding to the actual recording from the sweeps
        # and align sweeps so that the experiment would start at the same time
        long_square_sweeps.select_epoch("recording")
        long_square_sweeps.align_to_start_of_epoch("experiment")

        # find the start and end time of the stimulus
        # (treating the first sweep as representative)
        stim_start_index, stim_end_index = get_stim_epoch(long_square_sweeps.i[0])
        stim_start_time = long_square_sweeps.t[0][stim_start_index]
        stim_end_time = long_square_sweeps.t[0][stim_end_index]

        # build the extractors
        spfx = SpikeFeatureExtractor(start=stim_start_time, end=stim_end_time)
        sptfx = SpikeTrainFeatureExtractor(start=stim_start_time, end=stim_end_time)

        # run the analysis and print out a few of the features
        long_square_analysis = spa.LongSquareAnalysis(spfx, sptfx, subthresh_min_amp=-100.0)
        data = long_square_analysis.analyze(long_square_sweeps)

        specimen_dict = summarize_cell_ephys_features(data)
        mouse_data_dict[specimen_id] = specimen_dict
    except urllib.error.HTTPError as err:
        print(err.code)



In [ ]:
print("{} specimens extracted".format(len(mouse_data_dict)))
dfObj = pd.DataFrame(mouse_data_dict)
dfObj.to_csv('mouse_features_from_nwb.csv')

In [4]:
all_features = pd.read_csv("C:/Users/Allose/Documents/GitHub//NWB_IPFX_Allen/human_features_from_nwb.csv")

In [14]:
list1 = list(all_features.keys()[1:])

In [11]:
archive_url, file_manifest, experiment_metadata = get_archive_info(dataset="human")
human_specimen_id = list(experiment_metadata['cell_specimen_id'])

In [15]:
lst = []
for item in list1:
    lst.append(int(item))
    
lst

[541557114,
 569835804,
 569860121,
 571640763,
 571732727,
 576110753,
 576118161,
 576134298,
 591317875,
 591337995,
 591399440,
 591422059,
 591442366,
 592470583,
 592518575,
 592532014,
 592536009,
 592544683,
 592558182,
 592592260,
 592598069,
 592598599,
 595572609,
 595583062,
 596036401,
 596049425,
 596052266,
 596053384,
 596069208,
 596892878,
 596898838,
 601940782,
 601958678,
 602165685,
 602234640,
 611616275,
 611630983,
 611926705,
 642862180,
 642867575,
 652075403,
 653034602,
 653040833,
 653804581,
 653825043,
 653825083,
 653854241,
 654180841,
 665694337,
 665711442,
 665713811,
 666155913,
 672184275,
 672299031,
 672646275,
 672650722,
 674643161,
 674660116,
 676464469,
 676983527,
 676983694,
 677060360,
 677060617,
 677088033,
 677099224,
 680581022,
 680581271,
 685767564,
 685806309,
 685814252,
 685815936,
 689300528,
 689306818,
 689309060,
 689312221,
 689354518,
 689360253,
 689370040,
 695496425,
 695503873,
 695509869,
 695510727,
 695517111,
 695

In [22]:
list(set(human_specimen_id).difference(lst))

[794299406,
 720856118,
 774399058,
 720845918,
 576140393,
 591411338,
 767061167,
 707828926,
 770268354,
 701072592,
 770307286,
 567754968,
 758816993,
 758523114,
 783624438,
 758704376,
 793560315,
 768848167,
 767433014,
 674636128,
 758305130,
 732170615,
 571511167,
 793615764,
 680581533,
 592585119,
 689323433,
 674615724,
 642864559,
 677094878,
 721571303,
 758312441,
 732159495,
 732204556,
 737134157,
 770544209,
 786437731,
 767422070,
 686078633,
 674613931,
 695510699,
 596894384,
 786520776,
 541549258,
 707833551,
 717087449,
 770466523,
 738027231,
 805782243,
 732063466,
 596894444,
 793375469,
 811883255,
 660494076,
 774416149,
 770255641,
 696109866,
 650074936,
 596917084,
 700826482,
 596891511,
 770645883,
 695494556,
 695574435]